# Clustering Crypto

In [32]:
#pip install plotly_express
#pip install hvplot
#pip install plotly-express

In [31]:
# Initial imports
import numpy as np
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from matplotlib import pyplot as plt

# Import PCA from sci-kit learn
from sklearn.decomposition import PCA
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

### Deliverable 1: Preprocessing the Data for PCA

In [3]:
# Load the crypto_data.csv dataset.
file_path = Path("../Challenge/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col = 0)
print(crypto_df.shape)
crypto_df.head(10)

(1252, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [4]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
## list columns
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [6]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df["Algorithm"].notna()]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [7]:
# Remove the "IsTrading" column. 
#crypto_df.drop(columns = "IsTrading", inplace =True)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [8]:
#find null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [9]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [10]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [11]:
# Create a new DataFrame that holds only the cryptocurrencies names.
CoinName_df = pd.DataFrame(crypto_df["CoinName"])
CoinName_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [12]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)

In [13]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])

In [14]:
# Standardize the data with StandardScaler().
x_scaled = StandardScaler().fit_transform(x.values)

In [15]:
print(crypto_df.shape)
crypto_df.head(10)

(532, 5)


,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,True,PoW/PoS,4.199995e+01,42
404,Scrypt,True,PoW/PoS,1.055185e+09,532000000
1337,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethash,True,PoW,1.076842e+08,0
LTC,Scrypt,True,PoW,6.303924e+07,84000000
DASH,X11,True,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,True,PoW,1.720114e+07,0
ETC,Ethash,True,PoW,1.133597e+08,210000000
ZEC,Equihash,True,PoW,7.383056e+06,21000000


### Deliverable 2: Reducing Data Dimensions Using PCA

In [22]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3, random_state = 0)
x_pca = pca.fit_transform(x_scaled)
x_pca

array([[-0.3371053 ,  1.06507336, -0.5642437 ],
       [-0.32043926,  1.06546928, -0.56454984],
       [ 2.30795808,  1.74139527, -0.6197928 ],
       ...,
       [ 0.3235557 , -2.31656199,  0.43998244],
       [-0.13489796, -2.13189179,  0.27539924],
       [-0.28908906,  0.85103153, -0.24571237]])

In [17]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=x_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
print(pcs_df.shape)
pcs_df.head(10)

(532, 3)


,PC 1,PC 2,PC 3
42,-0.327166,0.914351,-0.574207
404,-0.310549,0.913838,-0.574720
1337,2.277918,1.272892,-0.738022
BTC,-0.145859,-1.330144,0.213128
ETH,-0.132538,-1.949282,0.443211
LTC,-0.146836,-1.053958,-0.003688
DASH,-0.425126,1.005767,-0.517726
XMR,-0.140443,-2.163629,0.310200
ETC,-0.130986,-1.949439,0.443165
ZEC,-0.183312,-1.911363,0.459464


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [21]:
# Create an elbow curve to find the best value for K.
import pandas as pd
#import plotly.express as px
import hvplot.pandas
from sklearn.cluster import KMeans

inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

#Plot elbow curve
df_elbow.hvplot.line(x="k", y="inertia",xticks=k, title="Elbow Curve")

C:\Users\Kumar\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [23]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=1)
# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [24]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join(pcs_df, how = "inner")


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, CoinName_df], axis=1)


#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 10)


,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,True,PoW/PoS,4.199995e+01,42,-0.327166,0.914351,-0.574207,42 Coin,1
404,Scrypt,True,PoW/PoS,1.055185e+09,532000000,-0.310549,0.913838,-0.574720,404Coin,1
1337,X13,True,PoW/PoS,2.927942e+10,314159265359,2.277918,1.272892,-0.738022,EliteCoin,1
BTC,SHA-256,True,PoW,1.792718e+07,21000000,-0.145859,-1.330144,0.213128,Bitcoin,0
ETH,Ethash,True,PoW,1.076842e+08,0,-0.132538,-1.949282,0.443211,Ethereum,0
LTC,Scrypt,True,PoW,6.303924e+07,84000000,-0.146836,-1.053958,-0.003688,Litecoin,0
DASH,X11,True,PoW/PoS,9.031294e+06,22000000,-0.425126,1.005767,-0.517726,Dash,1
XMR,CryptoNight-V7,True,PoW,1.720114e+07,0,-0.140443,-2.163629,0.310200,Monero,0
ETC,Ethash,True,PoW,1.133597e+08,210000000,-0.130986,-1.949439,0.443165,Ethereum Classic,0
ZEC,Equihash,True,PoW,7.383056e+06,21000000,-0.183312,-1.911363,0.459464,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [40]:
# Creating a 3D-Scatter with the PCA data and the clusters
import plotly_express as px
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
 )
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName','Algorithm', 'Proof ']

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
